In [1]:
from ecmwf.opendata import Client
import xarray as xr
import pandas as pd

from datetime import datetime
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))  # set vscode notebook path for module imports

from utils.tools import regions_from_xarray

In [2]:
steps = [i for i in range(0, 144, 3)] + [j for j in range(144, 361, 6)] #get all steps
time = 0 if datetime.now().hour < 18 else 12 #get 00z forecast if we are before 12z, else get 12z forecast
current_date = datetime.now().date() #select current date forecast
var = '2t' #forecast returned variables

In [3]:
#Download forecast data
client = Client()
result = client.retrieve(
    date=current_date, #can be 0 for today, -1 yestarday... if 
    time=time,
    type="fc",  #forecast for HRES
    stream='oper',
    step=steps,
    param=var,
    target="current.grib2",
)

To ensure the stability of our systems and to preserve resources for our operational activities (network, compute, etc.), access to the open-data portal is limited to 500 simultaneous connections. This limit helps us guarantee reliable service for our operational users, especially during periods of high demand. For added reliability, the open-data is replicated across AWS, Azure, and Google Cloud. If you experience difficulties accessing the portal directly, you can also retrieve the data from these cloud platforms.
                                                                

By downloading data from the ECMWF open data dataset, you agree to the terms: Attribution 4.0 International (CC BY 4.0). Please attribute ECMWF when downloading this data.


In [4]:
#Open the forecast dataset and format the datas
ds = xr.open_dataset('current.grib2')
forecast_date = ds.time.values #keep track of the forecast run date time
us = ds.sel(**{"latitude": slice(50, 24), "longitude": slice(-125, -67)}) #Slice to get only US 
us = (us - 273.15) * 1.8 + 32 #Convert kelvin to °F
us.attrs['units'] = '°F'

us = us.swap_dims({"step": "valid_time"})

#We slice the dataset with valid_times to avoid non complete day between 00z run (complete days) and 12z run (non complete days)
us = us.sel(valid_time=slice(pd.Timestamp(pd.Timestamp(ds.time.values).date()) + pd.Timedelta(days=1), #set the dataset at the start of the next day for our first window
                             pd.Timestamp(pd.Timestamp(ds.time.values).date()) + pd.Timedelta(days=14, hours=23))) #and just before the last valid_time

#Now that we start the first day at 00z for 00z run AND 12z run, resample hourly to daily
us_daily = us.resample(valid_time="1D").mean()

us_daily_hdd = (65 - us_daily).clip(min=0) #compute HDD

Ignoring index file 'current.grib2.5b7b6.idx' older than GRIB file


In [5]:
pop = xr.open_dataarray('../utils/files/population_regridded_025deg.nc')

In [6]:
hdd_weighted = us_daily_hdd * pop #Weight the hdd by population for each point in the grid and each valid_time

In [7]:
horizons = [(0,2, 'Day 1-3'), (3,6, 'Day 4-7'), (7,13, 'Day 8-14')]
hdd_list = []
for horizon in horizons:
    #Create horizon date for slicing
    first_date = hdd_weighted.valid_time.min().values
    start_date = first_date + pd.Timedelta(days=horizon[0])
    last_date = first_date + pd.Timedelta(days=horizon[1])

    hdd_weighted_horizon = hdd_weighted.sel(valid_time=slice(start_date, last_date)).sum(dim='valid_time') #sumed HDD for every grid point in the horizon

    #Make US mean
    us_horizon_mean = hdd_weighted_horizon.mean(dim=['latitude', 'longitude']) #Mean every point in the US to have one mean weighted HDD for the horizon
    hdd_list.append({'forecast_run_time': forecast_date, 'region': 'US Mean', 'horizon_start': start_date, 'horizon_end': last_date,
                    'horizon_label': horizon[2], 'forecast_HDD': us_horizon_mean.t2m.item()})
    
    #Make region means
    zone_means = regions_from_xarray(hdd_weighted_horizon)

    #For each zone in the horizon, make a new row of data
    for zone_name, zone_data in zone_means.items():
        hdd_list.append({'forecast_run_time': forecast_date, 'region': zone_name, 'horizon_start': start_date, 'horizon_end': last_date,
                         'horizon_label': horizon[2], 'forecast_HDD': zone_data.t2m.item()})
        
#Make a dataframe of values
hdd_horizon_df = pd.DataFrame(hdd_list)

In [8]:
hdd_horizon_df.head()

,forecast_run_time,region,horizon_start,horizon_end,horizon_label,forecast_HDD
0,2026-01-30,US Mean,2026-01-31,2026-02-02,Day 1-3,1.779145e+06
1,2026-01-30,Great Lakes,2026-01-31,2026-02-02,Day 1-3,4.983731e+06
2,2026-01-30,Columbia-Pacific Northwest,2026-01-31,2026-02-02,Day 1-3,6.902799e+05
3,2026-01-30,Missouri Basin,2026-01-31,2026-02-02,Day 1-3,4.797711e+05
4,2026-01-30,North Atlantic-Appalachian,2026-01-31,2026-02-02,Day 1-3,9.825774e+06


In [47]:
def compute_forcast_hdd(filepath, horizons):
    #Open the forecast dataset and format the datas
    ds = xr.open_dataset(filepath)
    forecast_date = ds.time.values #keep track of the forecast run date time
    us = ds.sel(**{"latitude": slice(50, 24), "longitude": slice(-125, -67)}) #Slice to get only US 
    us = (us - 273.15) * 1.8 + 32 #Convert kelvin to °F
    us.attrs['units'] = '°F'

    us = us.swap_dims({"step": "valid_time"})

    #We slice the dataset with valid_times to avoid non complete day between 00z run (complete days) and 12z run (non complete days)
    us = us.sel(valid_time=slice(pd.Timestamp(pd.Timestamp(ds.time.values).date()) + pd.Timedelta(days=1), #set the dataset at the start of the next day for our first window
                                pd.Timestamp(pd.Timestamp(ds.time.values).date()) + pd.Timedelta(days=14, hours=23))) #and just before the last valid_time

    #Now that we start the first day at 00z for 00z run AND 12z run, resample hourly to daily
    us_daily = us.resample(valid_time="1D").mean()

    us_daily_hdd = (65 - us_daily).clip(min=0) #compute HDD

    #Open population file reggrided to weather forecasts
    pop = xr.open_dataarray('../utils/files/population_regridded_025deg.nc')

    hdd_weighted = us_daily_hdd * pop #Weight the hdd by population for each point in the grid and each valid_time

    #Compute HDD for every horizons
    hdd_list = []
    for horizon in horizons:
        #Create horizon date for slicing
        start_date = min(horizon[0:2])
        last_date = max(horizon[0:2])

        hdd_weighted_horizon = hdd_weighted.sel(valid_time=slice(start_date, last_date)).sum(dim='valid_time') #sumed HDD for every grid point in the horizon

        #Make US mean
        us_horizon_mean = hdd_weighted_horizon.mean(dim=['latitude', 'longitude']) #Mean every point in the US to have one mean weighted HDD for the horizon
        hdd_list.append({'forecast_run_time': forecast_date, 'region': 'US Mean', 'horizon_start': pd.Timestamp(start_date), 'horizon_end': pd.Timestamp(last_date),
                        'horizon_label': horizon[2], 'forecast_HDD': us_horizon_mean.t2m.item()})
        
        #Make region means
        zone_means = regions_from_xarray(hdd_weighted_horizon)

        #For each zone in the horizon, make a new row of data
        for zone_name, zone_data in zone_means.items():
            hdd_list.append({'forecast_run_time': forecast_date, 'region': zone_name, 'horizon_start': pd.Timestamp(start_date), 'horizon_end': pd.Timestamp(last_date),
                            'horizon_label': horizon[2], 'forecast_HDD': zone_data.t2m.item()})
            
    #Make a dataframe of values
    hdd_horizon_df = pd.DataFrame(hdd_list)

    return hdd_horizon_df

In [48]:
def base_vs_forecast(filepath, base_filepath, horizons):
    #Load and format base HDD file
    us_base_hdd = pd.read_csv(base_filepath)
    us_base_hdd = us_base_hdd.rename(columns={'Unnamed: 0': 'doy'})
    us_base_hdd = us_base_hdd.set_index('doy')

    #Compute HDD from forecast file
    hdd_forecast = compute_forcast_hdd(filepath, horizons)

    #Add day of year columns to prepare the sum of base days
    hdd_forecast['doy_horizon_start'] = hdd_forecast['horizon_start'].dt.day_of_year
    hdd_forecast['doy_horizon_end'] = hdd_forecast['horizon_end'].dt.day_of_year

    #Prepare a list of horizon days with horizon label
    horizon_doy = list(set([(start, end, label) for start, end, label in zip(hdd_forecast['doy_horizon_start'].values, 
                                                                             hdd_forecast['doy_horizon_end'].values, 
                                                                             hdd_forecast['horizon_label'].values)]))
    
    #Sum the mean base hdd for each day in each horzion and each region
    hdd_horizon_list = []
    for hd in horizon_doy:
        us_base_hdd_horizon = us_base_hdd.loc[hd[0]:hd[1]]
        summed_base_hdd_horizon = pd.DataFrame(us_base_hdd_horizon.T.sum(axis=1).reset_index())
        summed_base_hdd_horizon = summed_base_hdd_horizon.rename(columns={'index': 'region', 0: 'sum_base_HDD'})
        summed_base_hdd_horizon['horizon_label'] = hd[2]
        hdd_horizon_list.append(summed_base_hdd_horizon)
    full_hdd_base_horizon = pd.concat(hdd_horizon_list)

    #Merge the sum base hdd and the forcast hdd on each region and each horizon
    hdd_forecast_base = pd.merge(hdd_forecast, full_hdd_base_horizon, how='inner', on=['region', 'horizon_label'])
    hdd_forecast_base['delta_forecast_base'] = hdd_forecast_base['forecast_HDD'] - hdd_forecast_base['sum_base_HDD'] #delta between forecast and base

    return hdd_forecast_base

In [65]:
from db.mongo import MongoWrapper
from pipeline.config import settings

def forecast_vs_forecast(current_forecast):
        #Get current forcast date from base_vs_forecast compute
        current_forecast_time = current_forecast['forecast_run_time'].unique()[0]
        previous_forecast_time = current_forecast_time - pd.Timedelta(hours=12)

        #Database connection
        mongo = MongoWrapper(settings.MONGO_URI, settings.MONGO_DB)
        db = mongo.collection(settings.MONGO_COLLECTION)
        
        #Find previous forecasts in db, continue only if not empty (meaning we have a previous forecast that match)
        query_res = db.find({'forecast_run_time': previous_forecast_time})
        previous_hdd = pd.DataFrame(list(query_res))
        if not previous_hdd.empty:
            previous_hdd = previous_hdd.drop('_id', axis=1)
            prev_forecast_horizon = list(set([(pd.Timestamp(start), pd.Timestamp(end), label) for start, end, label in zip(previous_hdd['horizon_start'].values, 
                                                                                    previous_hdd['horizon_end'].values, 
                                                                                    previous_hdd['horizon_label'].values)]))
            
        #Compute current forecast file with previous forecast time horizon
        current_hdd_with_prev_horizon_df = compute_forcast_hdd('current.grib2', prev_forecast_horizon)

        #Make sure that we have the same number of rows in both previous and current forecasts
        assert len(previous_hdd) == len(current_hdd_with_prev_horizon_df)

        #Format and merge and compute current - previous forecast
        previous_hdd = previous_hdd.rename(columns={'forecast_run_time': 'prev_forecast_run_time', 'forecast_HDD': 'prev_forecast_HDD'})
        current_and_prev_hdd = pd.merge(previous_hdd, current_hdd_with_prev_horizon_df, how='inner', on=['region', 'horizon_start', 'horizon_end', 'horizon_label'])
        current_and_prev_hdd['delta_current_forecast_to_prev_forecast'] = current_and_prev_hdd['forecast_HDD'] - current_and_prev_hdd['prev_forecast_HDD']
        delta_forecast_to_forecast = current_and_prev_hdd[['forecast_run_time', 'region', 'horizon_start', 'horizon_end', 'horizon_label', 'delta_current_forecast_to_prev_forecast']]
        delta_forecast_to_forecast = delta_forecast_to_forecast.rename(columns={'horizon_start': 'prev_horizon_start', 'horizon_end': 'prev_horizon_end'})

        return delta_forecast_to_forecast

In [70]:
def full_forecast(base_forecast_data, forecast_vs_forecast_data):
    full_hdd = pd.merge(base_forecast_data, forecast_vs_forecast_data, how='inner', on=['forecast_run_time', 'region', 'horizon_label']).reset_index(drop=True)
    return full_hdd

In [66]:
from datetime import timedelta
data_date = result.datetime
horizons = [(data_date.date() + timedelta(days=1), data_date.date() + timedelta(days=3), 'Day 1-3'),
            (data_date.date() + timedelta(days=4), data_date.date() + timedelta(days=7), 'Day 4-7'),
            (data_date.date() + timedelta(days=8), data_date.date() + timedelta(days=14), 'Day 8-14')]

In [67]:
base_and_forecast = base_vs_forecast('current.grib2', '../utils/files/base.csv', horizons)
forecast_and_forecast = forecast_vs_forecast(base_and_forecast)

Ignoring index file 'current.grib2.5b7b6.idx' older than GRIB file
Ignoring index file 'current.grib2.5b7b6.idx' older than GRIB file


In [71]:
df_hdd = full_forecast(base_and_forecast, forecast_and_forecast)

In [73]:
df_hdd.head()

,forecast_run_time,region,horizon_start,horizon_end,horizon_label,forecast_HDD,doy_horizon_start,doy_horizon_end,sum_base_HDD,delta_forecast_base,prev_horizon_start,prev_horizon_end,delta_current_forecast_to_prev_forecast
0,2026-01-30,US Mean,2026-01-31,2026-02-02,Day 1-3,1.779145e+06,31,33,2.875859e+05,1.491559e+06,2026-01-30,2026-02-01,-6.363584e+05
1,2026-01-30,Great Lakes,2026-01-31,2026-02-02,Day 1-3,4.983731e+06,31,33,5.842958e+05,4.399436e+06,2026-01-30,2026-02-01,-2.025188e+06
2,2026-01-30,Columbia-Pacific Northwest,2026-01-31,2026-02-02,Day 1-3,6.902799e+05,31,33,1.446409e+05,5.456390e+05,2026-01-30,2026-02-01,-2.617819e+05
3,2026-01-30,Missouri Basin,2026-01-31,2026-02-02,Day 1-3,4.797711e+05,31,33,7.000968e+04,4.097615e+05,2026-01-30,2026-02-01,-1.914042e+05
4,2026-01-30,North Atlantic-Appalachian,2026-01-31,2026-02-02,Day 1-3,9.825774e+06,31,33,1.024323e+06,8.801451e+06,2026-01-30,2026-02-01,-3.495451e+06
